In [1]:
#%load_ext autoreload
#%autoreload 1
#%aimport graph_description
import networkx as nx
import pysubgroup as ps
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
from graph_description.datasets import nx_read_attributed_graph
from graph_description.utils import prune_sparse_selectors
from graph_description.networkx_aggregation import SumAggregator, MeanAggregator, apply_aggregator    

In [3]:
G, df = nx_read_attributed_graph("citeseer")

In [4]:
searchspace = ps.create_selectors(df, ignore=['label'])
searchspace = [sel for sel in searchspace if "==0" not in str(sel)]

In [5]:
%%time
# do the actual propagation
df1 = apply_aggregator(SumAggregator, df, G, searchspace)
df2 = apply_aggregator((SumAggregator, MeanAggregator), df1, G)

init
prep done
init
prep done
CPU times: total: 6.38 s
Wall time: 6.54 s


In [6]:
#create the final dataframe & searchspace
total_df = pd.concat([df], axis=1)

In [7]:
total_df["label"]

0       1
1       4
2       5
3       5
4       0
       ..
3307    4
3308    3
3309    5
3310    1
3311    1
Name: label, Length: 3312, dtype: int8

In [8]:
#pd.isna(train_df).any(axis=1).sum()

In [9]:
train_df = total_df.drop("label", axis=1)
train_df.columns= list(map(str, train_df.columns))

In [10]:
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(train_df, total_df['label'], test_size=.2)

In [11]:
from xgboost import XGBClassifier, XGBRFClassifier

In [12]:
#bst = XGBClassifier(n_estimators=10, max_depth=2, learning_rate=1, objective='binary:logistic')
## fit model
#bst.fit(X_train, y_train)

In [13]:
#bst.score(X_test, y_test)

In [14]:
#from sklearn.linear_model import LogisticRegression
#clf_LR = LogisticRegression()
#cross_val_score(clf_LR, train_df, total_df['label'], cv=5)

## For comparison

From Prediction instability paper:
### Citeseer

GAT: Accuracy 69.04+-0.88

GCB: Accuracy 69.11+- 0.61


## xgboost

In [15]:
%%time
clf = XGBClassifier(n_estimators=10, max_depth=3, learning_rate=1, objective='binary:logistic')
cross_val_score(clf, train_df, total_df['label'], cv=5)

CPU times: total: 54.5 s
Wall time: 16.6 s


array([0.6907994 , 0.67119155, 0.67371601, 0.6918429 , 0.6858006 ])

In [16]:
%%time
clf = XGBClassifier(n_estimators=15, max_depth=3, learning_rate=1, objective='binary:logistic')
cross_val_score(clf, train_df, total_df['label'], cv=5)

CPU times: total: 1min 4s
Wall time: 18 s


array([0.70135747, 0.67722474, 0.68731118, 0.68882175, 0.69335347])

In [17]:
%%time
clf = XGBClassifier(n_estimators=20, max_depth=3, learning_rate=1, objective='binary:logistic')
cross_val_score(clf, train_df, total_df['label'], cv=5)

CPU times: total: 1min 10s
Wall time: 18.9 s


array([0.70135747, 0.67269985, 0.70392749, 0.69335347, 0.70090634])

In [18]:
%%time
clf_RF = XGBRFClassifier(n_estimators=20, max_depth=3, learning_rate=1, objective='binary:logistic')
cross_val_score(clf_RF, train_df, total_df['label'], cv=5)

CPU times: total: 1min 10s
Wall time: 20.6 s


array([0.63499246, 0.61689291, 0.63897281, 0.64803625, 0.64954683])

## sklearn

In [19]:
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB
import warnings

In [20]:
%%time
dummy_clf = DummyClassifier()
cross_val_score(dummy_clf, train_df, total_df['label'], cv=5)

CPU times: total: 7.02 s
Wall time: 6.65 s


array([0.21116139, 0.21116139, 0.21148036, 0.21299094, 0.21148036])

In [21]:
%%time
clf_NB = GaussianNB()
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.")
    cross_val_score(clf_NB, train_df, total_df['label'], cv=5)

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "E:\projects\graph_description\env\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "E:\projects\graph_description\env\lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "E:\projects\graph_description\env\lib\site-packages\sklearn\naive_bayes.py", line 263, in fit
    return self._partial_fit(
  File "E:\projects\graph_description\env\lib\site-packages\sklearn\naive_bayes.py", line 423, in _partial_fit
    X, y = self._validate_data(X, y, reset=first_call)
  File "E:\projects\gr

In [22]:
pd.isna(train_df).any(axis=1)

0       False
1       False
2       False
3       False
4       False
        ...  
3307    False
3308    False
3309    False
3310    False
3311    False
Length: 3312, dtype: Sparse[bool, False]